In [1]:
import torch
#import torchtext
#from torchtext.datasets import text_classification
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pandas as pd
import torch.utils.data as Data
import numpy as np
import torch.optim as optim
from torch.utils.data.dataset import random_split
import random
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# # 把为空的评论样本删除掉，存入train_sample.csv
# data = pd.read_csv('C:\\Users\\sheng\\Desktop\\Coronavirus\\sample_comment_seg.csv') 
# data_new = pd.DataFrame({ "comment_seg":[], "score":[]})
# for i in range(len(data)):
#     # data.loc[i,"comment_seg"].split()
#     if str(data.loc[i,"comment_seg"]) == "nan":
#         print("yes")
#     else:
#         data_new = data_new.append(data.loc[i,])
# data_new = data_new[["comment_seg", "score"]]
# data_new.to_csv('./train_sample.csv', sep=',', header=True, index=True, encoding='utf-8-sig')

# data["comment_length"] = None
# data["label"] = None
# label_dic = {-1: 2, 0: 0, 1: 1}

# for i in range(len(data)):
#     data.loc[i, "comment_length"] = len(data.loc[i,"comment_seg"].split())
#     data.loc[i, "label"] = label_dic.get(data.loc[i, "score"])

In [54]:
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [55]:
embed_dim = 200

In [56]:
inp = 'all_comments_seg_jieba.txt'
model_word2Vec = Word2Vec(LineSentence(inp), size=embed_dim, window=5, min_count=5, workers=multiprocessing.cpu_count())

In [57]:
data = pd.read_csv('train_sample.csv') 
data.head()

,Unnamed: 0,Unnamed: 0.1,comment_seg,score,comment_length,label
0,0,0,求求 那些 刷 车牌 的 能 不能 先 去 国务院 下面 的 疫情 监督 平台 先 去 举报...,1.0,72,1
1,1,1,任命 日报 就 可以 不 标明 出处,-1.0,7,2
2,2,2,回形针 牛 逼,1.0,3,1
3,3,3,二流 报纸 竟然 发 盗版 视频 啦,-1.0,7,2
4,4,4,我 在 武汉 看到 一只 鄂鱼 它 一声 尖叫 它 是 号 米 长 它 可恶 的 状 出入 ...,-1.0,28,2


In [58]:
def word2tensor(word):
    if word in model_word2Vec.wv.vocab.keys():
        v = torch.from_numpy(model_word2Vec.wv[word])
    else:
        # v = torch.randn(embed_dim)
        v = torch.zeros(embed_dim)
    return(v)
def comment_seg2tensor(comment_seg):
    comment_seg = comment_seg.split()
    t = torch.stack([word2tensor(word) for word in comment_seg])
    return(t)

In [59]:
BATCH_SIZE = 64
EMBED_DIM = embed_dim
HIDDEN_DIM = 16
OUTPUT_DIM = 3
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.2

In [60]:
class GetLoader(torch.utils.data.Dataset):
    # 初始化函数，得到数据
    def __init__(self, comment_seg, comment_length, label):
        self.comment_seg = comment_seg
        self.comment_length = comment_length
        self.label = label
        self.len = len(comment_seg)
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        return self.comment_seg[index], self.comment_length[index],self.label[index]
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一脸懵逼
    def __len__(self):
        return(self.len)

In [61]:
def generate_batch(batch):
    comments = [element[0] for element in batch]
    lengths = np.asarray([element[1] for element in batch])
    labels = np.asarray([element[2] for element in batch])
    MAX_LENGTH = max(lengths)
    sort_index = np.argsort(-lengths)
    text_tensor = torch.zeros([MAX_LENGTH, BATCH_SIZE, EMBED_DIM], dtype=torch.float)
    lengths_sorted = np.zeros(len(comments))
    labels_sorted = np.zeros(len(comments))
    for i in range(len(lengths)):
        text_tensor[0:lengths[sort_index[i]], i,:] = comment_seg2tensor(comments[sort_index[i]])
        lengths_sorted[i] = lengths[sort_index[i]] 
        labels_sorted[i] = labels[sort_index[i]]
    return text_tensor, torch.Tensor(lengths_sorted), torch.Tensor(labels_sorted)

In [62]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        
        super().__init__()
        
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text_tensor, text_lengths):
        
        #text = [sent len, batch size]
        
        # embedded = self.dropout(self.embedding(text))
        embedded = text_tensor
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        # hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)        
        #hidden = [batch size, hid dim * num directions]
        predictions = self.fc(hidden)
        return(predictions)

In [63]:
model = RNN(EMBED_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL,
            DROPOUT)

In [64]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

The model has 28,003 trainable parameters


In [65]:
# optimizer = torch.optim.SGD(model.parameters(), lr=1.0)
optimizer = optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss().to(device)
model = model.to(device)

In [66]:
def pred_accuracy(output_, label_):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 8
    """
    acc = (output_.argmax(1) == label_).sum()    
    return acc

In [67]:
def train(model, train_data_, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    c_seg = train_data_["comment_seg"]
    c_length =  train_data_["comment_length"]
    l = train_data_["label"]
    
    # 读取数据
    train_data_loader = GetLoader(c_seg, c_length, l)
    train_data_iterator = DataLoader(train_data_loader, batch_size=BATCH_SIZE, shuffle=True, drop_last=True,collate_fn=generate_batch)
    
    model.train()
    
    for i, batch in enumerate(train_data_iterator):
        text_tensor = batch[0].to(device)
        text_lengths = batch[1].to(device)
        labels = batch[2].to(device)
        optimizer.zero_grad()
                
        outputs = model(text_tensor, text_lengths).squeeze(1)
        
        loss = criterion(outputs, labels.long())
        
        acc = pred_accuracy(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_data_), epoch_acc / len(train_data_)

In [68]:
confusion_matrix = np.zeros([3,3])
confusion_matrix

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [69]:
def test(model, test_data_, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    c_seg = test_data_["comment_seg"]
    c_length =  test_data_["comment_length"]
    l = test_data_["label"]
    
    # 读取数据
    test_data_loader = GetLoader(c_seg, c_length, l)
    test_data_iterator = DataLoader(test_data_loader, batch_size=BATCH_SIZE, shuffle=True, drop_last=True,collate_fn=generate_batch)
    
    model.eval()
    confusion_matrix = np.zeros([3,3])
    
    with torch.no_grad():
    
        for i, batch in enumerate(test_data_iterator):
            text_tensor = batch[0].to(device)
            text_lengths = batch[1].to(device)
            labels = batch[2].to(device)
            optimizer.zero_grad()
                
            outputs = model(text_tensor, text_lengths).squeeze(1)
        
            loss = criterion(outputs, labels.long())
        
            acc = pred_accuracy(outputs, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            # 计算confusion matrix
            labels_pred = outputs.cpu().argmax(1)
            for (r,c) in zip(labels_pred, labels.cpu()):
                # print("r: ", r, "c: ", c)
                confusion_matrix[r.item(), int(c.item())] += 1

    return epoch_loss / len(test_data_), epoch_acc / len(test_data_), confusion_matrix

In [70]:
N_EPOCHS = 10

best_valid_loss = float('inf')

train_data_, test_data_, train_, test_ = train_test_split(data, range(len(data)),test_size=0.3, random_state=0)
train_data_ = pd.concat([train_data_["comment_seg"], train_data_["comment_length"], train_data_["label"]], axis = 1).reset_index(drop=True)
test_data_ = pd.concat([test_data_["comment_seg"], test_data_["comment_length"], test_data_["label"]], axis = 1).reset_index(drop=True)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_data_, optimizer, criterion)
    valid_loss, valid_acc, confusion_matrix = test(model, test_data_, optimizer, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print("confusion_matrix: ", confusion_matrix)

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.011 | Train Acc: 71.63%
	 Val. Loss: 0.009 |  Val. Acc: 77.66%
confusion_matrix:  [[4.800e+01 1.000e+00 1.000e+00]
 [6.600e+01 9.840e+02 2.340e+02]
 [3.720e+02 6.110e+02 3.443e+03]]
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.009 | Train Acc: 78.36%
	 Val. Loss: 0.008 |  Val. Acc: 78.71%
confusion_matrix:  [[  66.   11.   14.]
 [  61. 1021.  215.]
 [ 360.  564. 3448.]]
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.008 | Train Acc: 79.52%
	 Val. Loss: 0.008 |  Val. Acc: 78.84%
confusion_matrix:  [[  85.   16.   29.]
 [  59. 1037.  228.]
 [ 343.  542. 3421.]]
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.008 | Train Acc: 80.30%
	 Val. Loss: 0.008 |  Val. Acc: 79.26%
confusion_matrix:  [[  86.   12.   25.]
 [  63. 1106.  277.]
 [ 338.  478. 3375.]]
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.008 | Train Acc: 81.17%
	 Val. Loss: 0.008 |  Val. Acc: 79.45%
confusion_matrix:  [[  96.   20.   38.]
 [  57. 1081.  237.]
 [ 334.  496. 3401.]]
Epoch: 06

In [71]:
confusion_matrix

array([[ 115.,   28.,   71.],
       [  68., 1134.,  325.],
       [ 304.,  434., 3281.]])

In [72]:
np.sum(confusion_matrix,axis=0)

array([ 487., 1596., 3677.])

In [73]:
np.sum(confusion_matrix,axis=1)

array([ 214., 1527., 4019.])

In [75]:
3677/sum(np.sum(confusion_matrix,axis=0))

0.6383680555555555